In [41]:
from pathlib import Path

import fitz

import pandas as pd 
import re


In [86]:
def extrair_elementos_docling(caminho):
    doc = fitz.open(str(caminho))
    text = ""
    for page in doc:
       text += page.get_text()
    doc.close()
    return text

def parse_notas(texto):
    # Extrair cabeçalho
    turma = re.search(r"Turma\s+(\w+)", texto)
    disciplina = re.search(r"^Disciplina\s*$\n^(.+)$", texto, re.MULTILINE)
    ano_letivo = re.search(r"Ano Letivo\s+(\d{4})", texto)

    turma = turma.group(1) if turma else None
    disciplina = disciplina.group(1).strip() if disciplina else None
    ano_letivo = int(ano_letivo.group(1)) if ano_letivo else None

    # Expressão para capturar tudo de uma vez (aluno por aluno)
    padrao = re.compile(
        r"\b(\d+)\s+"                           # ordem (ignoramos)
        r"(\d{10}-\d)\s+"                       # matrícula
        r"([A-ZÀ-ÿ\s]+?)\s+"                    # nome (lazy)
        r"(-|[\d,]+)\s+"                        # nota 1
        r"(-|[\d,]+)\s+"                        # nota 2
        r"(-|[\d,]+)\s+"                        # nota 3
        r"(-|[\d,]+)\s+"                        # nota 4
        r"(-|[\d,]+)\s+"                        # para fechar
        r"(Aprovado|Reprovado|\(transferido\))",  # status
        flags=re.MULTILINE
    )

    dados = []
    for match in re.findall(padrao, texto):
        _, matricula, nome, n1, n2, n3, n4, fechar, status = match
        dados.append({
            "turma": turma,
            "ano_letivo": ano_letivo,
            "matricula": matricula,
            "aluno": nome.strip(),
            "disciplina": disciplina,
            "nota_1bim": float(n1.replace(',', '.')) if n1 != '-' else None,
            "nota_2bim": float(n2.replace(',', '.')) if n2 != '-' else None,
            "nota_3bim": float(n3.replace(',', '.')) if n3 != '-' else None,
            "nota_4bim": float(n4.replace(',', '.')) if n4 != '-' else None,
            "para_fechar": float(fechar.replace(',', '.')) if fechar != '-' else None,
            "status": status
        })

    return pd.DataFrame(dados)

In [47]:
path = Path().resolve()
current_path = path / "bases/notas/1A_ Mapa de notas_ Matemática.pdf"
current_path

WindowsPath('D:/Documentos/tcc - IA/projeto/bases/notas/1A_ Mapa de notas_ Matemática.pdf')

In [88]:
texto = extrair_elementos_docling(current_path)
df = parse_notas(texto)
df.to_csv("Notas_alunos.csv", index=False)

In [37]:
print(texto)

23/12/24, 09:43
Relatório de Fechamento de Notas
ESTADO DE GOIÁS SECRETARIA DE ESTADO DA EDUCAÇÃO 52033589 - CENTRO DE ENSINO EM PERÍODO INTEGRAL PROFESSOR JOAQUIM CARVALHO FERREIRA
Relatório de Fechamento de Notas por Disciplina
Data da Impressão: 23/12/2024
Ano Letivo/Semestre: 2024/2
Página 1
Composição de Ensino Ensino Médio Concomitante Intercomplementar à Educação Profissional - Técnico em Desenvolvimento em Web e Cibersegurança Turma Turno Disciplina
1A Matutino MATEMÁTICA
Série Data Emissão
1ª Série 23/12/2024
Ano Letivo
2024
Ord.
Matrícula ANA CLARA REIS DA SILVA 1 2412752317-6 ANA JULIA DA SILVA E SILVA 2 2412727121-1 ANNA CLARA CALERO GUIDA 3 1811397482-9 DÉRICK KAUÃ LIMA FERREIRA 4 2412732617-5 ELIAESTE GOMES SARDINHA NETO 5 2312711135-2 ELLEN ESTER DIAS FONSÊCA 6 1811400823-9 EVELYN GUEDES RAMOS 7 2312382937-0 FERNANDA SANTOS SILVA 8 2011829467-6 GABRIEL FERNANDO CARDIM OLIVEIRA FERNANDES 9 2412698289-2 GEANA MARIELY MIRANDA MORAES 10 1310202082-6 GEOVANNA SODRE CRUZ 11 23